In [1]:
#%load_ext autoreload
#%autoreload 2

#import sys
#sys.path.insert(0,'..')

import os

import backtrader as bt
import backtrader.indicators as btind
import numpy as np
import scipy.signal as signal

from gym import spaces

from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

from launcher import Launcher

In [ ]:
# GYM TEST ENV:
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_testing_gym',
)

env_config = dict(
    gym_id='Breakout-v0'
)


launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    train_steps=500000000,
    opt_learn_rate=1e-4,
    rollout_length=30,
    test_mode=True,
    model_summary_freq=50,
    episode_summary_freq=2,
    env_render_freq=10,
    verbose=0
    
)

In [ ]:
launcher.cluster_spec

In [ ]:
launcher.run()


In [2]:
class MyStrategy(BTgymStrategy):
    """
    Example subclass of BT server inner computation startegy.
    """
    
    def __init__(self, **kwargs):
        super(MyStrategy,self).__init__(**kwargs)
        
        self.order_penalty = 0.5
        self.trade_just_closed = False
        self.trade_result = None
        
    def notify_trade(self, trade):
        #if trade.justopened:
            #print('trade {} just opened'.format(trade.ref))
            
        if trade.isclosed:
            #print('trade {} closed, pnl_comm: {}'.format(trade.ref, trade.pnlcomm))
            # Set trade flag and result:
            self.trade_just_closed = True
            self.trade_result = trade.pnlcomm
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
        
    def get_state(self):
        """
        Computes featurized RL-ready environment observation state
        by applying continious wavelet transform to time-embedded vector
        of close-price gradients.
        """
        # Z-normalize raw inputs:
        mean = self.p.dataset_stat[1:3].values[0,:-1]
        std = self.p.dataset_stat[1:3].values[1,:-1]
        X = (self.raw_state - mean) / std
        #X = self.raw_state
        
        # Prepare parameters:
        Tau = 2
        max_cwt_scale = self.p.state_shape['model_input'].shape[1]
        cwt_width = np.linspace(Tau, max_cwt_scale + Tau - 1, max_cwt_scale) # scale of wavelet transdorm [n]
    
        
        # 'gamma'-like signal hyperparameter
        # for our signal to be in about [-5,+5] range before passing it to sigmoid;
        # tweak it by hand to add/remove "contrast":
        T = 2.5e+2
        
        # Get vector of gradients of last [n] prices:
        X = np.gradient(X, axis=0) * T
        
        # Use close price:
        channel = 3
        # Compute continious wavelet transform using Ricker wavelet, get [n,m,1]-dim. matrix:
        self.state['model_input'] = signal.cwt(X[:, channel], signal.ricker, cwt_width).T[:, :, None]
        
        # Squash values in [0,1]:
        self.state['model_input'] = self.sigmoid(self.state['model_input'])
        
        return self.state
    

    def R(self, val, val_start, R_max, DD_max):
        """
        Piecewise-linear normalized reward:
        val > 0         - values scalar or vector;
        val_start > 1   - start value, scalar;
        0 < R_max < 1   - maximum gain, percent/100,
        0 < DD_max < 1  - maximum draw-down, percent/100,
                          scalar (minimal acceptable value: = val_start*(1-DD_max) );
        returns vector in [-1,+1] range :
        """

        f_neg = np.asarray((val/val_start + DD_max - 1) / DD_max - 1)
        f_neg[f_neg > 0] = 0 
        f_neg[f_neg < -1] = -0.999
        f_pos = np.asarray((val/val_start - 1) / R_max)
        f_pos[f_pos < 0] = 0
        f_pos[f_pos > 1] = 1 

        return (f_neg + f_pos) / 2 +.5  # shift reward to [0,1] range
    
    def log_utility_R(self, val, val_start, R_max, DD_max, epsilon=0.001):
        """
        Normalized log utility reward function.
        Args:
            val > 0         - values scalar or vector;
            val_start > 1   - start value, scalar;
            0 < R_max < 1   - maximum gain, percent/100,
            0 < DD_max < 1  - maximum draw-down, percent/100,
                              scalar (minimal acceptable value: = val_start*(1-DD_max) );
            0 < epsilon <<1 - 'steepness' tuning parameter, scalar;
        Returns:
            piece-wise log utility in [0,1]
        """
        y = np.asarray((val/val_start + DD_max - 1)/(R_max + DD_max))
        y[y <= epsilon] = epsilon
        y[y > 1] = 1

        return 1 - np.log(y) / np.log(epsilon) 
    
    def power_utility_R(self, val, val_start, R_max, DD_max, power=2):
        """
        Normalized power utility Reward function.
        Args:
            val > 0         - values scalar or vector;
            val_start > 1   - start value, scalar;
            0 < R_max < 1   - maximum gain, percent/100,
            0 < DD_max < 1  - maximum draw-down, percent/100,
                              scalar (minimal acceptable value: = val_start*(1-DD_max) );
            1 < power       - 'steepness' tuning parameter, scalar;
        Returns:
            piece-wise power utility in [0,1]
        """
        y = np.asarray((val/val_start + DD_max - 1)/(R_max + DD_max))
        y[y < 0] = 0
        y[y > 1] = 1

        return y ** power
    
    def __get_reward(self):
        """
        Defines reward as [0,1]-bounded linear function of current to initial portfolio value ratio.
        """
        r = float(
            self.log_utility_R(
                val=self.env.broker.get_value(),
                val_start=self.env.broker.startingcash,
                R_max=self.p.target_call/100,
                DD_max=self.p.drawdown_call/100,
                epsilon=0.0001
            )
        )
        
        r /= 10 # normalize (kind of...)

        # Penalty for failed order:
        if self.order_failed:
            #print('Failed order!')
            r -= self.order_penalty
            self.order_failed = False
        
        return r 
    
    def get_reward(self):
        """
        Defines reward as [0,1]-bounded function of current trade result.
        """
        r = 0
        
        # Result
        if self.trade_just_closed:
            r = self.trade_result
            self.trade_just_closed = False
            
        # Penalty for failed order:
        if self.order_failed:
            #print('Failed order!')
            r -= self.order_penalty
            self.order_failed = False
            
        #print('reward_', r)
        
        return r / 20
            
# Set backtesting engine parameters:

state_shape = {
    'raw_state': spaces.Box(low=-1, high=1, shape=(60,4)),
    'model_input': spaces.Box(low=0, high=1, shape=(60, 30, 1))
}

MyCerebro = bt.Cerebro()
MyCerebro.addstrategy(MyStrategy,
                      state_shape=state_shape,
                      portfolio_actions=('hold', 'buy', 'sell'),
                      drawdown_call=5, # in percent
                      target_call=10,
                      skip_frame=10,
                     )

# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0)
MyCerebro.broker.set_shortcash(False)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10000,)


MyCerebro.addanalyzer(bt.analyzers.DrawDown)

# Provide data:
MyDataset = BTgymDataset(
    filename='../data/DAT_ASCII_EURUSD_M1_2016.csv',
    #filename='../data/test_sine_1min_period256_delta0002.csv',
    start_weekdays=[0, 1, 2, 3, 4],
    episode_len_days=0,
    episode_len_hours=23,
    episode_len_minutes=0,
    start_00=True,
    time_gap_hours=2,
)
env_config = dict(
    dataset=MyDataset,
    engine=MyCerebro,
    render_modes=['episode', 'human', 'model_input'],
    render_state_as_image=True,
    render_ylabel='Z-norm/CWT',
    render_size_episode=(12,8),
    render_size_human=(8, 3.5),
    render_size_state=(10, 5),
    render_dpi=75,
    port=5000,
    data_port=4999,
    connect_timeout=60,
    verbose=0,
)
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_testing',
)
launcher = Launcher(
    cluster_config=cluster_config,
    env_class=BTgymEnv,
    env_config=env_config,
    rollout_length=20,
    test_mode=False,
    train_steps=1000000000,
    model_summary_freq=10,
    episode_summary_freq=2,
    env_render_freq=10,
    verbose=1
    
)

In [3]:
launcher.run()

[2017-08-22 00:45:22,687] Press `Ctrl-C` to stop training and close launcher.


INFO:tensorflow:Restoring parameters from ./tmp/a3c_testing/train/model.ckpt-240


[2017-08-22 00:45:22,775] Restoring parameters from ./tmp/a3c_testing/train/model.ckpt-240


Press `Ctrl-C` to stop training and close launcher.
INFO:tensorflow:Starting standard services.


[2017-08-22 00:45:26,456] Starting standard services.


...

[2017-08-22 00:49:26,471] global/global_step/sec: 111.215


INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt


[2017-08-22 00:50:26,459] Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt


INFO:tensorflow:global/global_step/sec: 110.07


[2017-08-22 00:51:26,468] global/global_step/sec: 110.07
Process BTgymServer-2:2:
Process BTgymServer-3:1:
Process BTgymDataFeedServer-2:1:
[2017-08-22 00:51:36,131] worker_1 has joined.
[2017-08-22 00:51:36,135] chief_worker_0 has joined.
[2017-08-22 00:51:36,136] parameter_server_0 has joined.
[2017-08-22 00:51:36,137] Launcher closed.
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/dataserver.py", line 100, in run
    service_input = socket.recv_pyobj()
  File "/Users/muzikin/Yand

In [ ]:
print(launcher.kwargs, '\n\n')
print(launcher.env_config)
print(launcher.cluster_config)
print(launcher.cluster_spec)
for config in launcher.workers_config_list:
    print('============')
    for k, v in config.items():
        print('{}:\n{}\n'.format(k, v))
    

In [ ]:
def func1(max_step):
    step = 0
    done = False
    
    def func2(max_step):
        nonlocal step
        nonlocal done
        step +=1
        if step == max_step:
            step = 0
            done = True
        return step
    
    for i in range(20):
        done = False
        print(func2(max_step), step, done)
        


func1(7)
            

In [ ]:
a = dict()
a.update({'b': 2, 'c':4})
type(a) == dict